In [ ]:
# conda update -n base -c defaults conda
import warnings
warnings.filterwarnings("ignore")
import os
import scanpy as sc

import mudata as mu
import matplotlib.pyplot as plt
import arviz as az

import pertpy as pt

In [ ]:
adata_path = os.path.join("./clean_object_for_proportional_analysis.h5ad")
adata = sc.read(adata_path)
#adata

In [ ]:
#organize data
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(adata, type="cell_level", 
                                generate_sample_level=True, 
                                cell_type_identifier="cluster_celltype", 
                                sample_identifier="individual_ID", 
                                covariate_obs=["diagnosis","sex","age","PMI","regions","batch"])
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

## Try on running between regions first

In [ ]:
# Select control and eoad data focusing on specific region
sccoda_data.mod["coda_eoad"] = sccoda_data["coda"][sccoda_data["coda"].obs["regions"].isin(["HIP","EC","PFC"])].copy()

In [ ]:
pt.pl.coda.boxplots(sccoda_data, modality_key="coda_eoad", feature_name="regions", 
                    add_dots=True,dpi=500,figsize=(14, 6))

plt.savefig('./Figures/Brain_regions.pdf',dpi=500)
plt.show()


In [ ]:
# Stacked barplot for each sample
pt.pl.coda.stacked_barplot(sccoda_data, modality_key="coda_eoad", feature_name="individual_ID")
plt.show()

# Stacked barplot for the levels of "Condition"
pt.pl.coda.stacked_barplot(sccoda_data, modality_key="coda_eoad", feature_name="diagnosis")
plt.show()

# Stacked barplot for the levels of "Brain regions"
pt.pl.coda.stacked_barplot(sccoda_data, modality_key="coda_eoad", feature_name="regions")
plt.show()

In [ ]:
pt.pl.coda.rel_abundance_dispersion_plot(sccoda_data, modality_key="coda_eoad", abundant_threshold=0.9)

In [ ]:
sccoda_data = sccoda_model.prepare(sccoda_data, 
                                   modality_key="coda_eoad", 
                                   formula="C(regions, Treatment('PFC')) + C(diagnosis, Treatment('NCI'))", 
                                   reference_cell_type="OPC")
sccoda_data["coda_eoad"]

In [ ]:
# Run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda_eoad")
sccoda_data["coda_eoad"]
sccoda_model.summary(sccoda_data, modality_key="coda_eoad")
sccoda_model.credible_effects(sccoda_data, modality_key="coda_eoad")

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda_eoad", est_fdr=0.05)
sccoda_model.summary(sccoda_data, modality_key="coda_eoad")

In [ ]:
# saving
path = "test"
sccoda_data.write_h5mu(path)

# loading
sccoda_data_2 = mu.read_h5mu(path)

sccoda_model.summary(sccoda_data_2, modality_key="coda_eoad")

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data_2, modality_key="coda_eoad")
test_model.get_effect_df(sccoda_data, modality_key="coda_eoad")
test_model.get_effect_df(sccoda_data, modality_key="coda_eoad").to_csv("./Results/Peopoetional_analysis_regions_restuls.csv")

## run the analysis on each brain region, between EOAD and control

In [ ]:
# Select control and eoad data focusing on specific region
reion_oi="EC"
sccoda_data.mod["coda_eoad"] = sccoda_data["coda"][sccoda_data["coda"].obs["regions"].isin([reion_oi])].copy()
print(sccoda_data["coda_eoad"])

In [ ]:
#pt.pl.coda.boxplots(sccoda_data, modality_key="coda_eoad", feature_name="regions", 
#                    add_dots=True,dpi=500,figsize=(14, 6))

pt.pl.coda.boxplots(sccoda_data, modality_key="coda_eoad", feature_name="diagnosis", 
                    add_dots=True,dpi=500,figsize=(6, 4))

plt.savefig('./Figures/HIP.pdf',dpi=500)

plt.show()

In [ ]:
pt.pl.coda.rel_abundance_dispersion_plot(sccoda_data, modality_key="coda_eoad", abundant_threshold=0.9)
sccoda_data = sccoda_model.prepare(sccoda_data, 
                                   modality_key="coda_eoad", 
                                   formula="C(diagnosis, Treatment('NCI'))", 
                                   reference_cell_type="OPC")

#sccoda_data = sccoda_model.prepare(sccoda_data, 
#                                   modality_key="coda_eoad", 
#                                   formula="C(regions, Treatment('PFC')) + C(diagnosis, Treatment('NCI'))", 
#                                   reference_cell_type="OPC")
sccoda_data["coda_eoad"]

In [ ]:
# Run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda_eoad")
sccoda_data["coda_eoad"]

In [ ]:
sccoda_model.summary(sccoda_data, modality_key="coda_eoad")
sccoda_model.credible_effects(sccoda_data, modality_key="coda_eoad")
sccoda_model.set_fdr(sccoda_data, modality_key="coda_eoad", est_fdr=0.05)
sccoda_model.summary(sccoda_data, modality_key="coda_eoad")

In [ ]:
# saving
path = "test"
sccoda_data.write_h5mu(path)

# loading
sccoda_data_2 = mu.read_h5mu(path)

sccoda_model.summary(sccoda_data_2, modality_key="coda_eoad")

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data_2, modality_key="coda_eoad")
test_model.get_effect_df(sccoda_data, modality_key="coda_eoad")
test_model.get_effect_df(sccoda_data, modality_key="coda_eoad").to_csv("./Results/Peopoetional_analysis_HIP_restuls.csv")